In [1]:
import pandas as pd
import sqlite3

In [48]:
employees_df = pd.read_csv("data/employees.csv", parse_dates=["join_date", "resign_date"])
employees_df = employees_df.rename(columns={"employe_id": "employee_id"})
employees_df.head()


,employee_id,branch_id,salary,join_date,resign_date
0,1,3,7500000,2018-08-23,NaT
1,7,1,7500000,2017-04-28,NaT
2,8,1,13000000,2017-04-28,NaT
3,9,1,13500000,2017-12-22,2020-10-14
4,10,1,9000000,2018-03-01,2020-12-20


In [49]:
timesheets_df = pd.read_csv("data/timesheets.csv", parse_dates=["date"])

timesheets_df['checkin'] = pd.to_datetime(timesheets_df['checkin'], format='%H:%M:%S')
timesheets_df['checkout'] = pd.to_datetime(timesheets_df['checkout'], format='%H:%M:%S')


timesheets_df.head()
timesheets_df.shape[0]

39714

In [50]:
df = timesheets_df.merge(employees_df, how="left", on="employee_id")
df.head()

,timesheet_id,employee_id,date,checkin,checkout,branch_id,salary,join_date,resign_date
0,23907432,66,2019-08-21,1900-01-01 08:13:31,1900-01-01 17:05:02,2,10000000,2018-12-10,NaT
1,23907433,22,2019-08-21,1900-01-01 08:56:34,1900-01-01 18:00:48,1,8000000,2019-01-21,NaT
2,23907434,21,2019-08-21,1900-01-01 09:45:08,1900-01-01 18:24:06,1,13500000,2019-01-02,NaT
3,23907435,63,2019-08-21,1900-01-01 09:55:47,NaT,2,13500000,2018-07-30,2019-10-18
4,23907437,60,2019-08-21,1900-01-01 09:56:05,1900-01-01 17:31:08,2,12500000,2017-06-01,2020-09-04


In [27]:
# df["checoutin"] = (df['checkout'] - df['checkin']).apply(lambda x: x.seconds //3600)
# df[["timesheet_id", "checoutin"]].sort_values(by=["timesheet_id"])

,timesheet_id,checoutin
0,23907432,8.0
1,23907433,9.0
2,23907434,8.0
3,23907435,NaN
4,23907437,7.0
...,...,...
39787,70649593,8.0
39788,70649597,9.0
39789,70650186,NaN
39790,70650458,9.0


In [51]:
df.shape[0]

39792

In [55]:
df = df.dropna(subset=["date"])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['working_hours'] = (df['checkout'] - df['checkin']).apply(lambda x: x.seconds // 3600)
df['working_hours'] = df['working_hours'].fillna(8)
df['working_hours'].sum()

328530.0

In [59]:
result = df.groupby(['year', 'month', 'branch_id']).agg({'salary': 'sum', 'working_hours': 'sum'}).reset_index()
result['salary_per_hour'] = round(result['salary'] / result['working_hours'], 2)


In [57]:
result

,year,month,branch_id,salary,working_hours,salary_per_hour
0,2019,8,1,150500000,119.0,1264706.0
1,2019,8,2,252000000,163.0,1546012.0
2,2019,8,3,26000000,16.0,1625000.0
3,2019,9,1,1691000000,1488.0,1136425.0
4,2019,9,2,765000000,519.0,1473988.0
...,...,...,...,...,...,...
234,2020,12,2633,551500000,348.0,1584770.0
235,2020,12,2635,147000000,146.0,1006849.0
236,2020,12,3092,795000000,768.0,1035156.0
237,2020,12,11265,187000000,152.0,1230263.0


In [60]:
conn=sqlite3.connect('salary_2.db')

result[["year", "month", "branch_id", "salary_per_hour"]].to_sql(name="salary_per_hour", con=conn, if_exists='append', index=False)
conn.commit()


OperationalError: database is locked

In [ ]:
%load_ext sql
%sql sqlite:///salary_2.db

In [ ]:
%%sql 
SELECT * FROM salary_per_hour

Running query in 'sqlite:///salary_2.db'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(sqlite3.OperationalError) no such table: timesheets
[SQL: SELECT
COUNT(*),
SUM((strftime('%s', t.checkout) - strftime('%s', t.checkin)) / 3600)
FROM timesheets t]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

If you need help solving this issue, send us a message: https://ploomber.io/community
